# Домашнее задание 1: обработка твитов

Дедлайн: 03.10.2019

## Задание 1

Сколько твитов в наборе?

_**Решение 1:**_

(Комментировать тут нечего: решение было дано в конеспекте семинара)

In [1]:
import json

twits = []
for line in open('hw_3_twitter.json'):
    twits.append(json.loads(line))
print(len(twits))

2556


## Задание 2

Какой процент твитов составляют удаленные записи? (помеченные как deleted).

_**Решение 2:**_

Через документацию Twitter и редактор Notepad рассмотрим структуру данных в файле ещё раз.

В массиве twits каждый элемент - это словарь. 

Из всех пар первого уровня (потому что значение в паре само может быть словарём) нам нужны те, у которых ключи "delete".

Метод get() возвращает значение по указанному ключу. Если указанного ключа не существует, метод вернёт None и ненужная строка не попадёт в наш список удалённых записей.

In [2]:
deleted_twits = [] #удалённые твиты
exist_twits = [] # задел на задание 3: не-удалённые твиты

for line in twits:
    if line.get("delete") != None: 
        deleted_twits.append(line)
    else:
        exist_twits.append(line)  # задел на задание 3

percentage = (len(deleted_twits)/len(twits))*100
percentage = str(round(percentage, 2)) # запись с 2мя знаками после запятой аккуратней
print(percentage, "%")

14.16 %


## Задание 3

Какие самые популярные языки твитов?

_**Решение 3:**_

В прошлом задании мы создали список не-удалённых твитов exist_twits. Теперь будем пользоваться им постоянно! В частности, сейчас.

Из документации мы знаем, что в словаре-твите есть ключ "user". Значение этого ключа - словарь. В нём и содержится интересующая нас пара с ключом "lang" и значением в виде сокращённого обозначения языка.

In [51]:
lang_data = [] # информация о языках

for line in exist_twits:
    user_dict = line["user"]
    lang_data.append(user_dict.get("lang"))

from collections import Counter # частотный словарь

cntr = Counter(lang_data)
lang_list = cntr.most_common(15)

for item in lang_list: # для красивого вывода
    print(item[1]," ", item[0])

1048   en
411   ja
183   es
116   ko
104   th
80   pt
69   ar
39   fr
27   id
24   tr
22   ru
9   en-gb
8   de
8   it
7   nl


## Задание 4

Есть ли твиты от одного и того же пользователя? Если да, то сколько таких пользователей?

**_Решение 4:_**

Так как имена могут повторяться, а ID пользователя неудобно для восприятия, создаём их комбинированный список. Дальше - составляем частотный словарь упоминания этих комбинаций. Элементы частотного словаря проверяем на соответствие условию: если человек написал больше одного твита, то мы добавляем 1 в счётчик "num".

In [71]:
id_name_data = [] # вся идентифицирующая инфа

for line in exist_twits:
    user_dict = line["user"]
    id_ = user_dict.get("id_str") # достаём ID
    name = user_dict.get("name") # достаём имя
    id_name = id_, name # ОП! ID-имя С:
    id_name_data.append(id_name)

from collections import Counter # частотный словарь

cntr = Counter(id_name_data)
name_list = cntr.most_common() # список авторов, ранжированный по кол-ву их твитов

num = 0 
for item in name_list:
    if item[1] > 1:
        num = num + 1 # кол-во авторов, написавших более 1го твита
    
print(num)

25


## Задание 5

Топ-20 хэштегов (для них есть специальное поле).

_**Решение 5:**_

Алгоритм очень похож на решение 3го задания, но здесь вновь потребовалось обратиться к структуре файла. Выяснилось, что ключ "hashtags" содержит в себе не список, а словарь, и его нужно открыть с помощью ключа "text", чтобы вычленить наконец текст хэштэга.

In [13]:
hashtag_data = [] # все хэштэги



for line in exist_twits:
    user_dict = line["entities"] # другой ключ для словаря-твита
    hashtag = user_dict.get("hashtags") # ещё один словарь
    
    for item in hashtag:
        #print(item)
        hashtag_data.append(item.get("text"))

from collections import Counter # частотный словарь

cntr = Counter(hashtag_data)
hashtag_list = cntr.most_common(20)

for item in hashtag_list: # для красивого вывода
    print(item[1]," ", item[0])

17   BTS
13   방탄소년단
11   AMAs
8   人気投票ガチャ
7   태형
6   뷔
5   BTSinChicago
5   BTSLoveYourselfTour
5   오늘의방탄
5   PledgeForSwachhBharat
5   MPN
4   PCAs
4   V
4   시카고1회차공연
4   เป๊กผลิตโชค
4   JIMIN
3   running
3   NCT
3   지민
3   WajahmuPlastik


## Задание 6

Предобработать тексты оригинальных твитов (не ретвитов) на английском языке (убрать пунктуацию, привести к нижнему регистру) и составить частотный словарь.

**_Решение 6:_**

Для начала потребуется отсортировать оригинальные твиты от ретвитов. Далее - выбираем из оригинальных англоязычные. Затем вычленим из них текст, уровняем и почистим его, в завершение - частотный словарь.

In [49]:
original_twits = []
for line in exist_twits: # находим все оригинальные твиты (не-ретвиты)
    if line.get("retweeted_status") == None:
        original_twits.append(line)

en_twits = []    
for line in original_twits: # находим среди них англоязычные
    user_dict = line["user"]
    if user_dict.get("lang") == "en":
        en_twits.append(line) # получаем массив текстов
        
import re

words = []
for line in en_twits: # приводим каждый текст к нижнему регистру
    text = line.get("text")
    low_text = text.lower()
    
    word_list = re.findall(" ([a-z]+) ", low_text) # достаём только слова
    words.extend(word_list)

from collections import Counter # частотный словарь

cntr = Counter(words)
words_cntr = cntr.most_common(20)

for item in words_cntr: # для красивого вывода
    print(item[1]," ", item[0])

66   the
37   and
31   i
30   to
30   a
24   you
19   it
19   is
18   of
16   in
15   be
14   on
12   for
12   not
12   with
11   that
11   my
10   so
10   are
10   have


## Задание 7

Найти количество подписчиков (фолловеров) у авторов твитов и вывести топ-10.

**_Решение 7:_**

Задание похоже на задание 4, но теперь нам потребуется создать словарь с парами "имя" : "количество подписчиков", чтобы потом расставить его элементы в порядке убывания.

In [91]:
name_fcount_data = {} # создаём словарь

for line in exist_twits:
    user_dict = line["user"]
    name = user_dict.get("name")  
    fcount = user_dict.get("followers_count") 
    name_fcount_data[name] = [fcount] # складываем в словарь пары имя:кол-во подписчиков

from collections import Counter # частотный словарь

cntr = Counter(name_fcount_data)
popular_list = cntr.most_common(10)

for item in popular_list: # для красивого вывода
    print(item[1][0]," ", item[0])

2521403   Filosofía♕
1491309   FITNESS Magazine
1206759   malaysiakini.com
1137374   NYT Science
625463   Gramática
392472   TGRT Haber
383698   The Sun Football ⚽
374222   Melbourne, Australia
318189   Roznama Express
311319   💞 ცųཞɠɛཞცơơɠıɛ 💞


## Задание 8*

Топ-10 источников твита (из какого приложения написан) (подсказка: для обработки используйте регулярные выражения).

_**Решение 8:**_

Смотрим на документацию Твиттера, потом смотрим на то, как всё это выглядит в настоящем файле. Находим какие-то опорные объекты в строках (например, rel="nofollow">) и в своём регулярном выражении отражаем их, а тот кусочек строки, который нас непосредственно интересует, выделяем круглыми скобками.
Привычно складываем всё в списочек и делаем частотный словарь.

In [97]:
import re

source_list = []
for line in exist_twits:
    source_item = line.get("source")
    result = re.findall('rel="nofollow">(.+)</a>', source_item)
    source_list.extend(result)

from collections import Counter # частотный словарь

cntr = Counter(source_list)
sources = cntr.most_common(10)

for item in sources: # для красивого вывода
    print(item[1]," ", item[0])

800   Twitter for iPhone
695   Twitter for Android
140   Twitter Web Client
122   twittbot.net
51   Twitter Lite
28   Twitter for iPad
23   TweetDeck
17   Facebook
14   IFTTT
10   تطبيق قرآني
